
#### Step 1: Create Gold Schema

- 🏆 Creates the **`gold_ecommerce`** schema to store **curated, aggregated, and business-ready data**.
- 📊 This layer supports **analytics, dashboards, and reporting**, often consumed by business users and decision-makers.


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS dataengineer_databricks_ecommerce.gold_ecommerce
COMMENT 'Business-level curated and aggregated data for analytics and reporting';


#### Step 2: Create Gold Volume

- 📦 Creates the **`gold_volume`** inside the `gold_ecommerce` schema to **store final outputs and artifacts**.
- 🗃️ This volume is used for **business-ready datasets**, reports, and other deliverables used in analytics and decision-making.


In [0]:
%sql
CREATE VOLUME IF NOT EXISTS dataengineer_databricks_ecommerce.gold_ecommerce.gold_volume
COMMENT 'Volume for storing gold layer outputs and artifacts';


#### Step 3: Create and Query Gold Table

- 🏗️ Creates or replaces the `complete` table in the Gold schema using cleaned data from the Silver volume.
- 📂 Reads the data with `read_files()` directly from the volume path.
- 🔄 Keeps the table updated with the latest version of the data.
- 🔍 Shows the first 10 rows to confirm the data was loaded correctly.



In [0]:
%sql
CREATE OR REPLACE TABLE dataengineer_databricks_ecommerce.gold_ecommerce.complete
AS
SELECT * FROM read_files(
    "/Volumes/dataengineer_databricks_ecommerce/silver_ecommerce/silver_volume/orders_cleaned/",
    format => "csv"
);
select * from dataengineer_databricks_ecommerce.gold_ecommerce.complete limit 10;


#### Ex 1: Sales Summary by State and Category

This step creates a table that shows total sales, profit, and quantity for each state and product category. The data comes from cleaned CSV files in the Silver Layer.


In [0]:
%sql
CREATE OR REPLACE TABLE dataengineer_databricks_ecommerce.gold_ecommerce.sales_by_state_category
AS
SELECT State_Code,
       Category,
       sum(sales) as Total_sales,
       sum(profit) as Total_Profit,
       sum(Quantity) as Total_Quantity

FROM read_files(
    "/Volumes/dataengineer_databricks_ecommerce/silver_ecommerce/silver_volume/orders_cleaned/",
    format => "csv"
)
group by State_Code,Category
ORDER BY State_Code,Category;
select * from dataengineer_databricks_ecommerce.gold_ecommerce.sales_by_state_category


#### Note

This query uses the same logic as before, but now it reads data from the **Gold Layer** instead of the **Silver CSV files**.



In [0]:
%sql
CREATE OR REPLACE TABLE dataengineer_databricks_ecommerce.gold_ecommerce.sales_by_state_category_1
AS
SELECT State_Code,
       Category,
       sum(sales) as Total_sales,
       sum(profit) as Total_Profit,
       sum(Quantity) as Total_Quantity

FROM dataengineer_databricks_ecommerce.gold_ecommerce.complete
group by State_Code,Category
ORDER BY State_Code,Category;
select * from dataengineer_databricks_ecommerce.gold_ecommerce.sales_by_state_category_1


#### Step 2: Create Top Products by Sales Table

This query creates a **Gold Layer** table that ranks products based on total sales. It aggregates sales and profit for each product and assigns a rank using a window function.


In [0]:
%sql
create or replace table dataengineer_databricks_ecommerce.gold_ecommerce.Top_Products_By_Sales
as
select Product, 
       sum(Sales) as Total__Sales,
       sum(profit) as Total_profit,
       Rank() over (order by sum(sales) desc) as Rank
from dataengineer_databricks_ecommerce.gold_ecommerce.complete
group by Product;
select * from dataengineer_databricks_ecommerce.gold_ecommerce.Top_Products_By_Sales;

####Ex 3: Monthly Sales Trend Table

This query creates a table that shows total sales and profit for each month. It helps track how sales change over time.


In [0]:
%sql
create or replace table dataengineer_databricks_ecommerce.gold_ecommerce.Monthly_Sales
as
select DATE_FORMAT(DATE_TRUNC('month', Order_Date), 'yyyy-MM-dd') AS Month,
       sum(sales) as Total_Sales,
       sum(Profit) as Total_profit
from dataengineer_databricks_ecommerce.gold_ecommerce.complete
group by DATE_TRUNC('month', Order_Date)
order by Month;
    
select * from dataengineer_databricks_ecommerce.gold_ecommerce.Monthly_Sales

#### Ex 4: Supervisor Performance Table

This query creates a table that shows how each supervisor is performing. It includes total sales, total profit, and the number of orders handled.


In [0]:
%sql
create or replace table dataengineer_databricks_ecommerce.gold_ecommerce.Supervisor_Performance
as
select Assigned_Supervisor,
       sum(sales) as Total_Sales,
       sum(Profit) as Total_profit,
       count(Distinct Order_Number) as Total_Orders
from dataengineer_databricks_ecommerce.gold_ecommerce.complete
where Assigned_Supervisor is not null
group by Assigned_Supervisor;
    
select * from dataengineer_databricks_ecommerce.gold_ecommerce.Supervisor_Performance;